# Анализ пользователей мобильного приложения по продаже продуктов питания

На основании предоставленных данных необходимо разобраться, как ведут себя пользователи нашего мобильного приложения

**Наши задачи:**
1. Изучить воронку продаж:   
узнать как пользователи доходят до покупки;


2. Исследовать результаты A/A/B-эксперимента:  
дизайнеры захотели поменять шрифты во всём приложении, а менеджеры испугались, что пользователям будет непривычно, пользователей разбили на 3 группы: 2 контрольные со старыми шрифтами и одну экспериментальную — с новыми, необходоимо выяснить, какой шрифт лучше. 

**Описание данных:**

Каждая запись в логе — это действие пользователя, или событие.   
EventName — название события;  
DeviceIDHash — уникальный идентификатор пользователя;  
EventTimestamp — время события;  
ExpId — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.  

##  Открываем файл с данными и изучаем общую информацию

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import math as mth
import seaborn
import plotly as pt
from plotly import graph_objects as go

In [2]:
try:
    logs = pd.read_csv("C:/Users/mrsag/Desktop/Аналитик буткемп/Проекты/Сборный проект 2/logs_exp.csv", sep='\t')
except:
    logs = pd.read_csv('/datasets/logs_exp.csv', sep='\t')


In [3]:
display(logs.head(),logs.info(),logs.isna().sum(), logs.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   EventName       244126 non-null  object
 1   DeviceIDHash    244126 non-null  int64 
 2   EventTimestamp  244126 non-null  int64 
 3   ExpId           244126 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 7.5+ MB


,EventName,DeviceIDHash,EventTimestamp,ExpId
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248


None

EventName         0
DeviceIDHash      0
EventTimestamp    0
ExpId             0
dtype: int64

413

In [4]:
logs.shape

(244126, 4)

1. Необходимо перевести названия колонок в единый регистр, изменить тип данных в колонке дата и время
2. Пропусков нет, дубликаты - 413 (менее 1% от количества строк, можно удалить)

## Подготовим данные

In [1]:
logs.columns = ['event_name', 'device_id', 'event_time', 'group']
logs['event_time'] = pd.to_datetime(logs['event_time'], unit='s')
logs['date'] = logs['event_time'].dt.date
logs['date'] = logs['date'].astype('datetime64')
logs = logs.drop_duplicates().reset_index(drop=True)

NameError: name 'logs' is not defined

In [ ]:
display(logs.info())

In [ ]:
def aab_group(gr):
    if gr==246:
        return 'A1'
    elif gr==247:
        return 'A2'
    elif gr==248:
        return 'B'
    else:
        return 'UnknownGroup'

logs['group'] = logs['group'].apply(aab_group)
logs

Данные готовы к анализу, для удобства создали группы А1, А2 - контрольные, B - экспериментальная

## Изучим и проверим данные

In [ ]:
print(f'Всего в логе {len(logs.event_name)} событий.')
print(f'Всего пользователей в логе {len(logs.device_id.unique())}.')
print(f'В среднем на пользователя приходится {int(len(logs) / len(logs.device_id.unique()))} события.')

In [ ]:
logs_device = int(len(logs) / len(logs.device_id.unique()))

In [ ]:
zz = logs.groupby('device_id').agg({'event_name':'count'}).reset_index()
plt.boxplot(zz['event_name'])
plt.ylim(0, 100);

In [ ]:
date_max = logs['event_time'].max()
date_min = logs['event_time'].min()
print('Данные за период от {} до {}'.format(date_min, date_max))
print('Период составляет:', (date_max - date_min).days, 'дней.')

In [ ]:
plt.figure(figsize =(8, 6))
logs['event_time'].hist(bins=100)
plt.title('Гистограмма по дате и времени')
plt.xticks(rotation=45)
plt.xlabel('Дата')
plt.ylabel('Частота')
plt.grid()
plt.show()

In [ ]:
logs_new = logs.loc[logs['date'] > '2019-07-31']

In [ ]:
plt.figure(figsize =(8, 6))
logs_new['event_time'].hist()
plt.title('Гистограмма по дате и времени')
plt.xticks(rotation=45)
plt.xlabel('Дата')
plt.ylabel('Частота')
plt.grid()
plt.show()

У нас есть данные за 13 дней, но информативными являются данные тестирования с 01.08.2019.  
Дальнейший анализ будем проводить для данных за этот период

In [ ]:
print(f'Всего в логе {len(logs.event_name)} событий.')
print(f'Всего в логе {len(logs_new.event_name)} событий после очистки.')
print('Cобытий потеряно', round((len(logs['event_name']) - len(logs_new['event_name'])) / len(logs['event_name']) * 100, 2), '%')
print()


print(f'Всего пользователей в логе {len(logs.device_id.unique())}.')
print(f'Всего пользователей в логе после очистки {len(logs_new.device_id.unique())}.')
print('Пользователей потеряно', round(((len(logs['device_id'].unique()) - len(logs_new['device_id'].unique())) / len(logs['device_id'].unique())) * 100, 2), '%')
print()

print(f'В среднем на пользователя приходится {int(len(logs) / len(logs.device_id.unique()))} события.')
print(f'В среднем после очистки на пользователя приходится {int(len(logs_new) / len(logs_new.device_id.unique()))} события.')
print('Данных потеряно', round((((len(logs) / len(logs.device_id.unique()))- (len(logs_new) / len(logs_new.device_id.unique())))/(len(logs) / len(logs.device_id.unique())))* 100, 2), '%')

Мы потеряли около 1% данных, а это незначительно для продолжения анализа

In [ ]:
logs_new.groupby('group').agg({'device_id': 'nunique'})

У нас есть пользователи из всех трёх экспериментальных групп с примерно одинаковым количеством

## Изучаем воронку событий

In [ ]:
logs_new['event_name'].value_counts()

In [ ]:
fig = go.Figure(data=[go.Pie(labels=logs_new['event_name'].value_counts().index, values=logs_new['event_name'].value_counts(), \
                             title='Доли событий')])
fig.show()

In [ ]:
users_count = logs_new.groupby('event_name').agg({'device_id':'nunique'}).sort_values(by='device_id', ascending=False)
users_count['ratio'] = (users_count['device_id'] / logs_new['device_id'].nunique() * 100).round(2)
users_count


In [ ]:
logs_new = logs_new.query('event_name != "Tutorial"')

По полученным данным видно, что чаще всего пользователи видят появление главного экрана, реже всего пользователи проходили обучение по использованию приложения.  
Первые 4 события происходят в правильном порядке, только обучение должно быть на 1 месте. Но многие пользователи могут пропускать его и переходить на главный экран, поэтому не будем учитывать его при расчете воронки

In [ ]:
users_count_new = logs_new.groupby('event_name').agg({'device_id':'nunique'}).sort_values(by='device_id', ascending=False)
funnel = logs_new.groupby('event_name')['device_id'].nunique().sort_values(ascending=False).to_list()

In [ ]:
print(f'С главного экрана в каталог товаров перешло - {round(funnel[1]/funnel[0] *100, 2)} % клиентов.')
print(f'С каталога товаров в корзину перешло - {round(funnel[2]/funnel[1] *100, 2)} % клиентов.')
print(f'С корзины до экрана успешной оплаты перешло - {round(funnel[3]/funnel[2] *100, 2)} % клиентов.')

In [ ]:
print(f'При переходе с главного экрана в каталог товаров теряется  - {100 - int(funnel[1]/funnel[0] *100)} % клиентов.')
print(f'При переходе с каталога товаров в корзину - {100 - int(funnel[2]/funnel[1] *100)} % клиентов.')
print(f'При переходе с корзины до экрана успешной оплаты - {100 - int(funnel[3]/funnel[2] *100)} % клиентов.')


Больше всего пользователей теряется на первом этапе при переходе с главного экрана в каталог товаров

In [ ]:
fig = go.Figure(go.Funnel(
    y = funnel['event_name'],
    x = funnel['device_id'], 
    textinfo = 'value+percent previous'))
fig.update_layout(title='Воронка', title_x = 0.5)
fig.show()

In [ ]:
print(f'Доля пользователей, которая доходит от первого события до оплаты - {round(funnel[3]/funnel[0] *100,2)} % клиентов.')

По полученным данным видно, что чаще всего пользователи видят появление главного экрана, реже всего пользователи проходили обучение по использованию приложения, поэтому мы исключаем его из дальнейшего анлиза. Больше всего пользователей теряется на первом этапе при переходе с главного экрана в каталог товаров. Доля пользователей, которая доходит от первого события до оплаты - 47.7 % клиентов.

## Изучаем результаты эксперимента

In [ ]:
users_group = logs_new.groupby('group')['device_id'].nunique()
users_group['A1+A2'] = users_group['A1'] + users_group['A2']
users_group

In [ ]:
users_A1 = logs_new[logs_new['group'] == 'A1']['device_id'].nunique()
users_A2 = logs_new[logs_new['group'] == 'A2']['device_id'].nunique()
print('Количество пользователей в различных группах различается нa:',
      round((users_A2 - users_A1) / users_A1 * 100, 2), '%')

Количество пользователей в различных группах различается примерно на 1%, это допустимо для продолжения анализа

In [ ]:
group_test = logs_new.pivot_table(
    index='event_name', 
    columns='group', 
    values='device_id',
    aggfunc='nunique').sort_values(by='A1', ascending=False)

group_test = group_test.reset_index()
group_test['A1+A2'] = group_test['A1'] + group_test['A2']
group_test['all'] = group_test['A1+A2'] + group_test['B']

group_test['%_A1'] = (group_test['A1'] / users_group['A1'] * 100).round(2)
group_test['%_A2'] = (group_test['A2'] / users_group['A2'] * 100).round(2)
group_test['%_B'] = (group_test['B'] / users_group['B'] * 100).round(2)
group_test['%_A1+A2'] = (group_test['A1+A2'] / users_group['A1+A2'] * 100).round(2)

group_test

In [ ]:
def z_test(group1, group2, alpha):
    for i in group_test.index:
        p1 = group_test[group1][i] / users_group[group1]
        p2 = group_test[group2][i] / users_group[group2]
        p_combined = ((group_test[group1][i] + group_test[group2][i]) / (users_group[group1] + users_group[group2]))
        difference = p1 - p2
        z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / users_group[group1] + 1 / users_group[group2]))
        distr = st.norm(0, 1) 
        p_value = (1 - distr.cdf(abs(z_value))) * 2
        n=16
        bonferroni_alpha = alpha / n
        print('{} p-значение: {}'.format(group_test['event_name'][i], p_value))
        if p_value < alpha:
            print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
            print()
        else:
            print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")
            print()

Нулевая гипотеза: между долями нет значимой разницы;  
Альтернативная гипотеза: между долями есть значимая разница;  
Порогом статистической значимости установим alpha = 0.01

*A1/A2 - эксперимент:*

In [ ]:
z_test("A1", "A2", 0.01)

Тест не зафиксировал разницы между группами A1/A2

Так как группы A1/A2 должны быть одинаковыми был взят уровень статистической значимости равный 1%. Для дальнейших экспериментов выберем самый стандартный уровень значимости равный 5%

*A1/B - эксперимент:*

In [ ]:
z_test("A1", "B", 0.05)

Тест не зафиксировал разницы между группами A1/B

*A2/B - эксперимент:*

In [ ]:
z_test("A2", "B", 0.05)

Тест не зафиксировал разницы между группами A2/B

*A1+A2/B - эксперимент:*

In [ ]:
z_test("A1+A2", "B", 0.05)

Тест не зафиксировал разницы между группами A1+A2/B

По результатам тестов можно сказать, что значимой разницы между группами во всех тестах не выявлено

**Вывод:**  
1. Мы использовали данные за период от 25.07.2019 04:43:36 до 07.08.2019 21:15:17, но так как первая неделя была неинформативной, мы продолжили анализ по данным с 01.08.2019
2. По полученным данным видно, что чаще всего пользователи видят появление главного экрана, реже всего пользователи проходили обучение по использованию приложения. Многие пользователи могут пропускать его и переходить на главный экран, поэтому не будем учитывать его при расчете воронки
3. Больше всего пользователей теряется на первом этапе при переходе с главного экрана в каталог товаров, доля пользователей, которая доходит от первого события до оплаты - 47.7 % клиентов
4. По результатам А/А/В-теста не было выявлено различий между контрольными и эксперементальной группами, что говорит о том, что замена шрифтов не влияет на пользователей